The intention of this project, is to identify how we can find an alternative route for a path.

The data being used is route information, with the intention to have machine learning figure out an alternative option for a route.

### 1. Transforming the Openflights Dataset

OpenFlights has a wealth of information about airports that span the globe. They also have a pretty nice data dictionary available

Source: https://openflights.org/data.php

In [2]:
with open('../../data/openFlightsRaw/zOpenFlightsDataDictionary.txt') as file:
    lines = file.readlines()
    for line in lines: 
        print(line, end='')

Airline Database
        As of January 2012, the OpenFlights Airlines Database contains 5888 airlines. 
        Each entry contains the following information:
        ---
        Airline ID      |       Unique OpenFlights identifier for this airline.
        Name	        |       Name of the airline.
        Alias	        |       Alias of the airline. For example, All Nippon Airways is commonly known as "ANA".
        IATA	        |       2-letter IATA code, if available.
        ICAO	        |       3-letter ICAO code, if available.
        Callsign	|       Airline callsign.
        Country	        |       Country or territory where airport is located. See Countries to cross-reference to ISO 3166-1 codes.
        Active	        |       "Y" if the airline is or has until recently been operational, "N" if it is defunct. This field is not reliable: in particular, major airlines that stopped flying long ago, but have not had their IATA code reassigned (eg. Ansett/AN), will incorrectly show

In [1]:
#necessary imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import googlemaps
import kaleido
import plotly.io as pio
pio.renderers.default = "iframe"

From the OpenFlights datasets, the part that we care about the most is airports and routes.

airports contains all of the information about airports throughout the world.
routes contains information about eligable routes between airports

In [2]:
airports_csv = pd.read_csv('../data/openFlightsRaw/airports.csv')
routes_csv = pd.read_csv('../data/openFlightsRaw/routes.csv')

Among routes and airports there is desire to create a new dataset that has elements from both flat files.

In [3]:
airports_csv.head(5)

,AIRPT_ID,NAME,CTY,CTRY,IATA,ICAO,LAT,LONG,ALT,TZ,DST,DBTZ,TYPE,SRC
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


In [4]:
routes_csv.head(5)

,AIRLINE,AIRLINE_ID,SRC_AIRPT,SRC_AIRPT_ID,DESTN_AIRPT,DESTIN_AIRPT_ID,CDSHARE,STOPS,EQPT
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2


To create an all encompassing routes file, I want a single data source to have the name, country, city, latitude, and longitude for source and destination airports.

In [5]:
#Verification that all of the identifier values are not null
print(routes_csv['SRC_AIRPT_ID'].isnull().values.any())
print(routes_csv['DESTIN_AIRPT_ID'].isnull().values.any())
print(airports_csv['AIRPT_ID'].isnull().values.any())

#Verification that no rows are duplicated in the routes file
print(routes_csv.duplicated().any())

False
False
False
False


In [6]:
#Identifying type differences between the two airport_csv and routes_csv dataframes
print(type(routes_csv['SRC_AIRPT_ID'][0]))
print(type(routes_csv['DESTIN_AIRPT_ID'][0]))
print(type(airports_csv['AIRPT_ID'][0]))

<class 'str'>
<class 'str'>
<class 'numpy.int64'>


In [7]:
#Turning AIRPT_ID into a string for joining purposes
airports_csv['AIRPT_ID'] = (airports_csv['AIRPT_ID']).astype(str)

My initial assumption about null identifiers was incorrect for routes, instead. As the data dictionary mentioned '\N' is used for null values, instead of just missing data.
This caused issues when originally joining the dataset.

In [8]:
#Define a null value
nullVal = r"\N"

#Remove the null value from the dataframes
routes_csv = routes_csv[routes_csv['SRC_AIRPT_ID'] != nullVal]
routes_csv = routes_csv[routes_csv['DESTIN_AIRPT_ID'] != nullVal]

I had assumed that the ids were in numerical order, and didn't have any holes.  That assumption was wrong, what this means there are airports in the routes data, that are not in the airports data.

In the chart below, you can see how the IDs for the last 1000 elements in our dataset jump from 10,000 to 14,000.

In [9]:
airports_csv_Example = pd.read_csv('../data/openFlightsRaw/airports.csv')
airports_csv_Example = airports_csv_Example.tail(1000)
airportIdPoints = np.array(airports_csv_Example['AIRPT_ID'])

fig = px.line(airports_csv_Example, 
              x = airports_csv_Example.index.values, 
              y = 'AIRPT_ID',
              title = 'Airport IDs are not in numerical order',
              labels = {'x' : 'Last 1000 Airport Rows', 'AIRPT_ID' : 'Airport ID'}
            )

fig.show()

In [10]:
#Identifying size of routes data, to ensure size stays the same or is less afterwards
len(routes_csv.index)

67240

Joining source and destination airports onto the routes.

Using inner joins gets rid of the airports that aren't in the airports.csv

This provides a name that the Google API can use, so distance can be obtained

In [11]:
routeDistSrc = routes_csv.merge(airports_csv, how='inner', left_on='SRC_AIRPT_ID', right_on='AIRPT_ID')

routeDistSrc['SRCNAME'] = routeDistSrc['NAME']
routeDistSrc['SRCCTY'] = routeDistSrc['CTY']
routeDistSrc['SRCCTRY'] = routeDistSrc['CTRY']
srcCols = ['AIRLINE', 'AIRLINE_ID', 'SRC_AIRPT', 'SRC_AIRPT_ID', 'SRCNAME', 'SRCCTY', 'SRCCTRY', 'DESTN_AIRPT', 'DESTIN_AIRPT_ID', 'CDSHARE', 'STOPS', 'EQPT']
routeDistSrc = routeDistSrc[srcCols]

routeDistDestin = routes_csv.merge(airports_csv, how='inner', left_on='DESTIN_AIRPT_ID', right_on='AIRPT_ID')

routeDistDestin['DESTINNAME'] = routeDistDestin['NAME']
routeDistDestin['DESTINCTY'] = routeDistDestin['CTY']
routeDistDestin['DESTINCTRY'] = routeDistDestin['CTRY']
destinCols = ['AIRLINE', 'AIRLINE_ID', 'SRC_AIRPT', 'SRC_AIRPT_ID', 'DESTN_AIRPT', 'DESTIN_AIRPT_ID', 'DESTINNAME', 'DESTINCTY', 'DESTINCTRY', 'CDSHARE', 'STOPS', 'EQPT']
routeDistDestin = routeDistDestin[destinCols]

routeMergeCols = ['AIRLINE', 'AIRLINE_ID', 'SRC_AIRPT', 'SRC_AIRPT_ID', 'DESTN_AIRPT', 'DESTIN_AIRPT_ID', 'CDSHARE', 'STOPS', 'EQPT']
routeDist = routeDistSrc.merge(routeDistDestin, how='inner', on=routeMergeCols)

In [12]:
#Verified there are less rows now, due to inner join
len(routeDist.index)

66771

In [13]:
# Saving off the combined dataframe
routeDist.to_csv('../data/routesDist.csv')

routeDist.head(5)

,AIRLINE,AIRLINE_ID,SRC_AIRPT,SRC_AIRPT_ID,SRCNAME,SRCCTY,SRCCTRY,DESTN_AIRPT,DESTIN_AIRPT_ID,CDSHARE,STOPS,EQPT,DESTINNAME,DESTINCTY,DESTINCTRY
0,2B,410,AER,2965,Sochi International Airport,Sochi,Russia,KZN,2990,NaN,0,CR2,Kazan International Airport,Kazan,Russia
1,2B,410,ASF,2966,Astrakhan Airport,Astrakhan,Russia,KZN,2990,NaN,0,CR2,Kazan International Airport,Kazan,Russia
2,2B,410,ASF,2966,Astrakhan Airport,Astrakhan,Russia,MRV,2962,NaN,0,CR2,Mineralnyye Vody Airport,Mineralnye Vody,Russia
3,2B,410,CEK,2968,Chelyabinsk Balandino Airport,Chelyabinsk,Russia,KZN,2990,NaN,0,CR2,Kazan International Airport,Kazan,Russia
4,2B,410,CEK,2968,Chelyabinsk Balandino Airport,Chelyabinsk,Russia,OVB,4078,NaN,0,CR2,Tolmachevo Airport,Novosibirsk,Russia


### 2. Trying to Reduce the Dataset

With the intention of getting distance information, Google's Distance Matrix API can be used.

There are a couple of issues though
1. Hitting the API over and over is costly and takes time.
2. The API calculates distance with driving, so utilizing routes overseas doesn't make much sense.

I will start by seeing which airports have the most traffic, this will help me use the distance calculation for common routes with future work.

With the graph below there appears to be a linear relation in how often the airport is used as a source and a destination within the dataset.

In [14]:
routeDist['countVal'] = 1
srcAIRPTS = routeDist.groupby('SRCNAME')['countVal'].sum()
destinAIRPTS = routeDist.groupby('DESTINNAME')['countVal'].sum()

airportCnt = airports_csv.merge(srcAIRPTS, how='inner', left_on='NAME', right_on='SRCNAME')
airportCnt = airportCnt.rename(columns={'countVal': 'SRCCNT'})
airportCnt = airportCnt.merge(destinAIRPTS, how='inner', left_on='NAME', right_on='DESTINNAME')
airportCnt = airportCnt.rename(columns={'countVal': 'DESTINCNT'})

fig = px.scatter(airportCnt, 
              x = 'SRCCNT', 
              y = 'DESTINCNT',
              title = 'Airports have Traffic Coming In and Out Linearly',
              labels = {'SRCCNT' : 'Source of Route Count', 'DESTINCNT' : 'Destination of Route Count'}
            )

fig.show()

With the graph above, we can observe that there is an outlier.

The most popular airport is Hartsfield Jackson Atlanta International Airport

In [15]:
airportCnt[airportCnt['SRCCNT'] > 800]

,AIRPT_ID,NAME,CTY,CTRY,IATA,ICAO,LAT,LONG,ALT,TZ,DST,DBTZ,TYPE,SRC,SRCCNT,DESTINCNT
1806,3682,Hartsfield Jackson Atlanta International Airport,Atlanta,United States,ATL,KATL,33.6367,-84.428101,1026,-5,A,America/New_York,airport,OurAirports,915,911


We can use the data above to figure out which countries have the most route data. 
It looks like China and the United States have the most route data 

In [16]:
srcCtryCnt = airportCnt.groupby('CTRY')['SRCCNT'].sum()

#Filtering out countries that sources less than 500 flights
srcCtryCntPi = srcCtryCnt[srcCtryCnt > 500]

fig = px.pie(srcCtryCntPi,
             names = srcCtryCntPi.index,
             values = srcCtryCntPi.values,
             title = "United States Sources the largest amount of flights")

fig.show()

Removing some of the outlier values and looking at the top highest countries with source routes, to verify source and destination linear trend

In [17]:
ctrys = ['United States', 'China', 'Spain', 'United Kingdom', 'Germany']

airportCntFiltered = airportCnt[airportCnt['CTRY'].isin(ctrys)]
airportCntFiltered = airportCntFiltered[airportCntFiltered['SRCCNT'] < 500]
airportCntFiltered = airportCntFiltered[airportCntFiltered['SRCCNT'] > 100]

fig = px.scatter(airportCntFiltered, 
              x = 'SRCCNT', 
              y = 'DESTINCNT',
              title = 'The Linear Relationship Between Airports Routing In and Out',
              color = airportCntFiltered['CTRY'],
              labels = {'SRCCNT' : 'Source of Route Count', 'DESTINCNT' : 'Destination of Route Count', 'CTRY' : 'Country'}
            )

fig.show()

Based on these results, we can safely assume that United States Data would be ideal to focus on

### 3. Adding Distance for the US

As identified through the previous EDA, the source and destination count should be the same.

In [18]:
#Utilizing the file that was created earlier
routes_csv = pd.read_csv('../data/routesDist.csv')

usSrcRoutes = routes_csv[routes_csv['SRCCTRY'] == 'United States']
print(len(usSrcRoutes))

usDestinRoutes = routes_csv[routes_csv['DESTINCTRY'] == 'United States']
print(len(usDestinRoutes))

13021
13016


In [19]:
#Merging the destination and source routes to ideall capture data that includes international sources
mergeCols = ['AIRLINE_ID', 'SRC_AIRPT_ID', 'DESTIN_AIRPT_ID']

usRoutes = usSrcRoutes.merge(usDestinRoutes, how='outer', on=mergeCols)
len(usRoutes)

15519

In [20]:
#Verification that there are no duplicates amongst the data
usRoutesTest = pd.concat([usSrcRoutes,usDestinRoutes])
usRoutesTest = usRoutesTest.drop_duplicates()
len(usRoutesTest)

15519

In [21]:
#re-using variable above that was used to show concat and outer join provide similar data
usRoutes = usRoutesTest[['SRC_AIRPT_ID', 'SRCNAME', 'DESTIN_AIRPT_ID', 'DESTINNAME']]

In [22]:
#A function to capture the data from Google's distance matrix API
def distCalc(x, y):
    file = open("../dataEngineering/gMapsAPIKey.txt")
    gMapsKey = file.read()
    file.close()

    gmaps = googlemaps.Client(key=gMapsKey)

    distance = gmaps.distance_matrix(x, y)['rows'][0]['elements'][0]

    try:
        meters = distance['distance']['value']
    except:
        meters = r'\N'

    try:
        seconds = distance['duration']['value']
    except:
        seconds = r'\N'

    return(meters,seconds)

The function above was ran through a for loop from the distanceEDA.ipynb in order to create a dataset of seconds and meters that could be used.

It could only run through 100 routes at a time, so the process took quite a few hours.

The desire was to follow the rule the data dictionary set-up, and use '\N' as a value for null characters

The output of that process was used to create a dataset called usRoutes that includes meters and seconds.

In [23]:
permSeriesM = pd.read_csv('../data/meters.csv')
permSeriesS = pd.read_csv('../data/seconds.csv')

secondsSeries = pd.Series(permSeriesS.to_numpy().flatten())
metersSeries = pd.Series(permSeriesM.to_numpy().flatten())

usRoutes["seconds"] = secondsSeries
usRoutes["meters"] = metersSeries

#usRoutes.to_csv('../data/usRoutes.csv')

C:\Users\thoma\AppData\Local\Temp\ipykernel_24796\894388009.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\thoma\AppData\Local\Temp\ipykernel_24796\894388009.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [24]:
usRoutes = pd.read_csv('../data/usRoutes.csv')

usRoutes.head(5)

,Unnamed: 0,og_index,SRC_AIRPT_ID,SRCNAME,DESTIN_AIRPT_ID,DESTINNAME,seconds,meters
0,0,166,3531,Kodiak Airport,7162,Larsen Bay Airport,0,0
1,1,167,7162,Larsen Bay Airport,7161,Karluk Airport,\N,\N
2,2,248,5726,Southeast Iowa Regional Airport,3830,Chicago O'Hare International Airport,\N,\N
3,3,249,5726,Southeast Iowa Regional Airport,3678,St Louis Lambert International Airport,13361,386142
4,4,250,4042,Decatur Airport,3830,Chicago O'Hare International Airport,11518,323800


### 4. Performing EDA on usRoutes

The purpose behind this first graph was to discover whether or not every airport had a route that had traffic going in and out.

For most of the time, this was true.  There were a couple cases in South America and Africa that ended up causing the data to show up oddly.

In [25]:
airports = pd.read_csv('../data/openFlightsRaw/airports.csv')

mapDat = usRoutes.merge(airports, how="left", left_on='SRCNAME', right_on='NAME')
mapDat = mapDat.rename(columns={'LAT': 'SRCLAT', 'LONG': 'SRCLONG'})

mapDat = mapDat.merge(airports, how="left", left_on='DESTINNAME', right_on='NAME')
mapDat = mapDat.rename(columns={'LAT': 'DESTINLAT', 'LONG': 'DESTINLONG'})

srcFig = go.Figure(data=go.Scattergeo(
    name = "Has Outgoing Traffic",
    lat = mapDat['SRCLAT'].tolist(),
    lon = mapDat['SRCLONG'].tolist(), 
    mode = 'markers',
    opacity=0.70,
    marker = dict(
        size = 5,
        color = 'blue',
        symbol = 'triangle-up',
        standoff = 3
    )
))

destinFig = go.Figure(data=go.Scattergeo(
    name = "Has Incoming Traffic",
    lat = mapDat['DESTINLAT'].tolist(),
    lon = mapDat['DESTINLONG'].tolist(), 
    mode = 'markers',
    opacity=0.70,
    marker = dict(
        size = 5,
        color = 'red',
        symbol = 'triangle-down'
    )
))

fig = go.Figure(data=go.Scattergeo())
fig.add_traces(srcFig._data)
fig.add_traces(destinFig._data)

fig.update_layout(
    title_text='All US Airports have Incoming and Outgoing Traffic',
    showlegend=True,
    geo=dict(
        scope = 'world',
        showland = True,
        landcolor = 'lightgray',
    )
)

fig.show()

Next the intention was to find the data shape of distance and time for routes.

In [26]:
import plotly.express as px

usRoutes = pd.read_csv('../data/usRoutes.csv')

nullVal = r"\N"
usRoutes = usRoutes[usRoutes['meters'] != nullVal]

usRoutes['meters'] = usRoutes['meters'].astype(int)
usRoutes['seconds'] = usRoutes['seconds'].astype(int)

usRoutes['hours'] = round(usRoutes['seconds']/3600, 0)

usRoutes['countVal'] = 1
usRoutesHours = usRoutes.groupby('hours')['countVal'].sum()
usRoutesHours = pd.DataFrame(usRoutesHours)
usRoutesHours = usRoutesHours.reset_index()

fig = px.bar(usRoutesHours, x='hours', y='countVal',
              labels = {
                  'countVal': "Count",
                  'hours': 'Time (hour)'
              },
              title = "Majority of Routes are within 20 hours by car")
fig.show()

usRoutes['km'] = round(usRoutes['meters']/1000, -2)
usRoutesKM = usRoutes.groupby('km')['countVal'].sum()
usRoutesKM = pd.DataFrame(usRoutesKM)
usRoutesKM = usRoutesKM.reset_index()

fig2 = px.bar(usRoutesKM, x = 'km', y = 'countVal',
              labels = {
                  'countVal': "Count",
                  'km': 'Distance (km)'
              },
              title = "Majority of Routes are within 2.5 Mm")
fig2.show()

With the ability to remove values from the dataset based on them being considered unachievable by driving, not having a value for meters or seconds, the dataset can become smaller.

Below is a visualization showing the routes that are unachieveable vs the achieveable routes.

The visualizations use opacity, so the darker markers are airports that typically see more traffic.  They are pretty similar among both of the visualizations.

In [27]:
import plotly.graph_objects as go

airports = pd.read_csv('../data/openFlightsRaw/airports.csv')
usRoutes = pd.read_csv('../data/usRoutes.csv')

mapDat = usRoutes.merge(airports, how="left", left_on='SRCNAME', right_on='NAME')
mapDat = mapDat.rename(columns={'LAT': 'SRCLAT', 'LONG': 'SRCLONG'})

mapDat = mapDat.merge(airports, how="left", left_on='DESTINNAME', right_on='NAME')
mapDat = mapDat.rename(columns={'LAT': 'DESTINLAT', 'LONG': 'DESTINLONG'})

nullVal = r"\N"
mapDat = mapDat[mapDat['meters'] == nullVal]

fig = go.Figure()

for index, row in mapDat.iterrows():

    fig.add_trace(go.Scattergeo(
        mode = "markers+lines",
        lat = [row['SRCLAT'], row['DESTINLAT']],
        lon = [row['SRCLONG'], row['DESTINLONG']],
        opacity=0.01,
        marker = dict(
            size = 10,
            color = 'black'
    )
        ))

fig.update_layout(
    title_text='All of the Unachieveable Routes by Car from Airport to Airport',
    showlegend=False,
    geo=dict(
        scope = 'usa',
        showland = True,
        landcolor = 'lightgray',
    )
)

fig.show()

In [28]:
import plotly.graph_objects as go

airports = pd.read_csv('../data/openFlightsRaw/airports.csv')
usRoutes = pd.read_csv('../data/usRoutes.csv')

mapDat = usRoutes.merge(airports, how="left", left_on='SRCNAME', right_on='NAME')
mapDat = mapDat.rename(columns={'LAT': 'SRCLAT', 'LONG': 'SRCLONG'})

mapDat = mapDat.merge(airports, how="left", left_on='DESTINNAME', right_on='NAME')
mapDat = mapDat.rename(columns={'LAT': 'DESTINLAT', 'LONG': 'DESTINLONG'})

nullVal = r"\N"
mapDat = mapDat[mapDat['meters'] != nullVal]

fig = go.Figure()

for index, row in mapDat.iterrows():

    fig.add_trace(go.Scattergeo(
        mode = "markers+lines",
        lat = [row['SRCLAT'], row['DESTINLAT']],
        lon = [row['SRCLONG'], row['DESTINLONG']],
        opacity=0.01,
        marker = dict(
            size = 10,
            color = 'black'
    )
        ))

fig.update_layout(
    title_text='All of the Achieveable Routes by Car from Airport to Airport',
    showlegend=False,
    geo=dict(
        scope = 'usa',
        showland = True,
        landcolor = 'lightgray',
    )
)

fig.show()

To verify the airports are pretty similar amongst the pictures, we can see the top few airports that source most of the routes.

In [29]:
usRoutes = pd.read_csv('../data/usRoutes.csv')

nullVal = r"\N"
usRoutes = usRoutes[usRoutes['meters'] == nullVal]

usRoutes['countVal'] = 1
usRoutesAirlines = usRoutes.groupby('SRCNAME')['countVal'].sum()
usRoutesAirlines = pd.DataFrame(usRoutesAirlines)
usRoutesAirlines = usRoutesAirlines.reset_index()

usRoutesAirlines = usRoutesAirlines[usRoutesAirlines['countVal'] >= 50]
fig = px.bar(usRoutesAirlines, x='SRCNAME', y='countVal',
              labels = {
                  'countVal': "Count",
                  'SRCNAME': 'Airports'
              },
              title = "Airports that Source 50 or more Routes")
fig.show()

Picking one of the airports from above, O'Hare, we can focus a visualization on just routes that have destinations to Chicago.

This can help identify potential alternative routes that could be taken to O'Hare.

In [30]:
import plotly.graph_objects as go

airports = pd.read_csv('../data/openFlightsRaw/airports.csv')
usRoutes = pd.read_csv('../data/usRoutes.csv')

mapDat = usRoutes.merge(airports, how="left", left_on='SRCNAME', right_on='NAME')
mapDat = mapDat.rename(columns={'LAT': 'SRCLAT', 'LONG': 'SRCLONG'})

mapDat = mapDat.merge(airports, how="left", left_on='DESTINNAME', right_on='NAME')
mapDat = mapDat.rename(columns={'LAT': 'DESTINLAT', 'LONG': 'DESTINLONG'})

nullVal = r"\N"
mapDat = mapDat[mapDat['meters'] != nullVal]
mapDat = mapDat[(mapDat['meters']).astype(int) < 1000000]
mapDat = mapDat[mapDat['SRCNAME'] == 'Chicago O\'Hare International Airport']

fig = go.Figure()

for index, row in mapDat.iterrows():

    fig.add_trace(go.Scattergeo(
        mode = "markers+lines",
        lat = [row['SRCLAT'], row['DESTINLAT']],
        lon = [row['SRCLONG'], row['DESTINLONG']],
        opacity=0.5,
        marker = dict(
            size = 10,
            color = 'blue'
    )
        ))

fig.update_layout(
    title_text='Possible destinations to Chicago O\'Hare International Airport',
    showlegend=False,
    geo=dict(
        scope = 'usa',
        showland = True,
        landcolor = 'lightgray',
    )
)

fig.show()

### 5. KMeans Clustering

After doing some research on reinforcement learning, it seemed like it would be an awesome way to solve our problem. The problem shortly became trying to figure out how to figure this out within a semester. 

After further research and a better understanding of a couple of the algorithms from class I wanted to focus on clustering, to see if I can natural create a group of airports that could work as alternative routes.

In [31]:
from sklearn.cluster import KMeans

airports_csv = pd.read_csv('../data/openFlightsRaw/airports.csv')

# Creating a dataset that just has latitude and longitudes, to use as data to feed kMeans
airportsData = airports_csv
airportsData = airportsData[airportsData['CTRY'] == 'United States']

airportsCoords = airportsData[['LAT', 'LONG']]
airportsCoords = airportsCoords.to_numpy()

With a dataset of points to cluster, I need to use the Elbow method to figure out what inertia value may provide an optimal amount of clusters.

It is important to note that this is being done with only airports in the United States, which was decided to be done from the previous EDA work. The importance of only focusing on the United States, allows the clusters to not have to worry about datapoints across borders or in the ocean.

In [32]:
inertiaList = []
kList = range(1, 50)

for k in kList:
    kMeans = KMeans(n_clusters = k, random_state = 0, n_init = 10).fit(airportsCoords)
    inertiaList.append(kMeans.inertia_)

With 50 different points, below is the Elbow method from that latitude and longitude data.

In [33]:
inertiaDF = pd.DataFrame(inertiaList, columns=['Inertias'])
fig = px.scatter(inertiaDF, y="Inertias")
fig.show()

One thing I noticed was how large the initial inertias were compared to, the rest of the points.

I decided to use a trick from data visualization, where I squared every point to ideally create a more defined curve.

In [34]:
inertiaDF['InertiasSquared'] = np.sqrt(inertiaDF['Inertias'])
fig = px.scatter(inertiaDF, y="InertiasSquared")
fig.show()

This resulted in a more ideal curve, following my understanding of the Elbow Method, I chose value 8 for the inertia, since it seemed like the bottom of the curve.

In [35]:
airportsData['cluster'] = KMeans(n_clusters = 8, random_state = 0, n_init = 10).fit_predict(airportsCoords)

#Mapping colors to the clusters
airportsData['clusterColor'] = airportsData['cluster'].map(pd.Series(px.colors.qualitative.Dark24))

C:\Users\thoma\AppData\Local\Temp\ipykernel_24796\1555511578.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\thoma\AppData\Local\Temp\ipykernel_24796\1555511578.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



The graph below, definately had regions of the United States clustered, but it wasn't what I expected for figuring out an alternative route.

My understanding of the inertias, they can be used to figure out the distance of the cluster from the centroid point. So I wanted to go with a higher inertia

In [36]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scattergeo(
    lat = airportsData['LAT'].tolist(),
    lon = airportsData['LONG'].tolist(), 
    mode = 'markers',
    marker = dict(
        size = 3,
        color = airportsData['clusterColor'].tolist(),
    ),
))

fig.update_layout(
    title_text='Clusters with Inertia 8 from KMeans',
    showlegend=True,
    geo=dict(
        scope = 'usa',
        showland = True,
        landcolor = 'lightgray',
    )
)

fig.show()

In [37]:
airportsData['cluster'] = KMeans(n_clusters = 22, random_state = 0, n_init = 10).fit_predict(airportsCoords)

airportsData['clusterColor'] = airportsData['cluster'].map(pd.Series(px.colors.qualitative.Dark24))

C:\Users\thoma\AppData\Local\Temp\ipykernel_24796\1974205059.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\thoma\AppData\Local\Temp\ipykernel_24796\1974205059.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



22 was picked for the inertia, since it was a point where the inertia squared value didn't seem to change by a degree of 5 or more anymore.

The results were more satisfying, but Alaska having 4 different clusters that seemed to be split evenly down the middle made me doubt the results.

In [38]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scattergeo(
    name = 'string',
    lat = airportsData['LAT'].tolist(),
    lon = airportsData['LONG'].tolist(), 
    mode = 'markers',
    marker = dict(
        size = 3,
        color = airportsData['clusterColor'].tolist(),
    ),
))

fig.update_layout(
    title_text='Clusters with Inertia 8 from KMeans',
    showlegend=True,
    geo=dict(
        scope = 'usa',
        showland = True,
        landcolor = 'lightgray',
    )
)

fig.show()

### 6. Density Clustering

Through some more research, it seems like Density clustering would solve my Alaska problem. Additionally I think it would help shape better clusters around a populated area, since there generally tends to be more airports in dense populations. 

One of the last things majority of the population wants to do is drive for over an hour after a long flight.

Similarly with KMeans, I wanted to figure out what was the best value to use for epsilon in the DBSCAN algorithm.

With some research, I couldn't find much in the way of an elbow method, but I did find an article stating do divide x amount of kilometers by 6371 when dealing with data across the globe

Source: https://geoffboeing.com/2014/08/clustering-to-reduce-spatial-data-set-size/

That source also recommended converting coordinates to radians, and using the haversine metric.  It also used the ball-tree algorithm, but when I was testing different results with that it didn't seem to change my results.

In [39]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps = 1/6371, min_samples = 10, metric = 'haversine').fit((np.radians(airportsCoords)))
print(np.unique(dbscan.labels_))

[-1]


Looking at the unique clusters created from using 1 km divided by 6371, it was pretty lackluster.

There was one cluster, and the value was -1, which was DBSCAN's way of saying every value is an outlier.

This meant, that my value was too small or too big, so I decided to try something similar to the elbow method. The following computes the amount of clusters every value would have between .005 and 1

In [40]:
itemList = []
epsilonList = np.arange(0.005, 1, 0.005)

for e in epsilonList:
    dbscan = DBSCAN(eps = e, min_samples = 10, metric = 'haversine').fit((np.radians(airportsCoords)))
    itemList.append(len(np.unique(dbscan.labels_)))

Most of the clusters made it seem like I have now overfitted the model. There was some significant data before using an epsilon of .055.

Additionally, 1/6371 ends up being 0.00015, so I would want to try a size smaller than that since .055 was on the larger side.



In [41]:
itemDF = pd.DataFrame({'Epislon': epsilonList, 'Clusters': itemList})
fig = px.scatter(itemDF, y = 'Clusters', x = 'Epislon')
fig.show()

In [42]:
itemList = []
epsilonList = np.arange(0.00001, 0.055, 0.00001)

for e in epsilonList:
    dbscan = DBSCAN(eps = e, min_samples = 10, metric = 'haversine').fit((np.radians(airportsCoords)))
    itemList.append(len(np.unique(dbscan.labels_)))

Using values between 0.00001 and 0.055, achieved much better results, where I decided to pick an epsilon of 0.01569, which generates 24 clusters.

In [43]:
itemDF = pd.DataFrame({'Epislon': epsilonList, 'Clusters': itemList})
fig = px.scatter(itemDF, y = 'Clusters', x = 'Epislon')
fig.show()

In [44]:
airportsData['densityCluster'] = DBSCAN(eps = 0.01569, min_samples = 10, metric = 'haversine').fit_predict((np.radians(airportsCoords)))

#Keep in mind -1 represents noise, and doesn't end up getting clustered, so transforming them into 0
airportsData['densityCluster'] = airportsData['densityCluster'] + 1
airportsData['densityClusterColor'] = airportsData['densityCluster'].map(pd.Series(px.colors.qualitative.Dark24))

C:\Users\thoma\AppData\Local\Temp\ipykernel_24796\3829614065.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\thoma\AppData\Local\Temp\ipykernel_24796\3829614065.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\thoma\AppData\Local\Temp\ipykernel_24796\3829614065.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

My Alaska issue was kind of fixed, which might be because 0.01569 is too strict of an epsilon. The resulting graph does show that the more densely populated areas had clusters that pertained their areas.

A good example is California, the Bay Area and Southern California areas were clustered separately.

In [45]:
fig = go.Figure()

fig.add_trace(go.Scattergeo(
    name = 'string',
    lat = airportsData['LAT'].tolist(),
    lon = airportsData['LONG'].tolist(), 
    mode = 'markers',
    opacity=0.70,
    marker = dict(
        size = 3,
        color = airportsData['densityClusterColor'].tolist()
    )
))

fig.update_layout(
    title_text='Clusters with DBSCAN with Outliers',
    showlegend=True,
    geo=dict(
        scope = 'usa',
        showland = True,
        landcolor = 'lightgray',
    )
)

fig.show()

The following graph just shows a cleaner picture of the DBSCAN results without outliers.

In [46]:
airportsDataDBScan = airportsData

airportsDataDBScan = airportsDataDBScan[airportsDataDBScan['densityCluster'] != 0]

fig = go.Figure()

fig.add_trace(go.Scattergeo(
    name = 'string',
    lat = airportsDataDBScan['LAT'].tolist(),
    lon = airportsDataDBScan['LONG'].tolist(), 
    mode = 'markers',
    opacity=0.70,
    marker = dict(
        size = 3,
        color = airportsDataDBScan['densityClusterColor'].tolist()
    )
))

fig.update_layout(
    title_text='Clusters with DBSCAN without Outliers',
    showlegend=True,
    geo=dict(
        scope = 'usa',
        showland = True,
        landcolor = 'lightgray',
    )
)

fig.show()

### 7. Comparing KMeans and DBSCAN

The DBSCAN method ended up creating what I was looking for more, but it immensely reduced the data that we would be using. While the California example shows that density clustering worked really well, the northern east coast had a different story. KMeans did not work with that area either, which drives me to a different conclusion.

The results from the first kmean cluster can be used to create regions, then each region cluster can use DBSCAN to figure out an epsilon that works well for that specific region. Additionally, I am working on finding some population data that could act as an additional factor for clustering rather than just using latitude and longitudes.


As far as results go, I decided to focus on the Washington cluster, which was cluster 6. Most of the destinations to the Washington airport go to the Seattle-Tacoma International Airport, which is ID 3577. So I removed all of those rows, and noticed that Bellingham International Airport was the only other airport in that cluster with destinations from the routes data. One of the sources that goes to Bellingham was Seattle-Tacoma. Which means I can say that there is a possible route incase the direct route to Seattle Tacoma isn't available.

In [47]:
washingtonCluster = usRoutes[usRoutes['DESTIN_AIRPT_ID'].isin(list(airportsDataDBScan[airportsDataDBScan['densityCluster'] == 6]['AIRPT_ID']))]

nullVal = r"\N"
washingtonCluster = washingtonCluster[washingtonCluster['seconds'] != nullVal]

washingtonCluster[washingtonCluster['DESTIN_AIRPT_ID'] != 3577]

,Unnamed: 0,og_index,SRC_AIRPT_ID,SRCNAME,DESTIN_AIRPT_ID,DESTINNAME,seconds,meters
2986,2986,11358,3728,Daniel K Inouye International Airport,3777,Bellingham International Airport,69184,2174892
3238,3238,11616,3577,Seattle Tacoma International Airport,3777,Bellingham International Airport,110692,3457980
6643,6643,28478,6505,Phoenix-Mesa-Gateway Airport,3777,Bellingham International Airport,63057,1949107
6773,6773,28608,3877,McCarran International Airport,3777,Bellingham International Airport,73630,2302614
6812,6812,28647,3484,Los Angeles International Airport,3777,Bellingham International Airport,66234,2023907
6844,6844,28679,3453,Metropolitan Oakland International Airport,3777,Bellingham International Airport,79387,2491829
6925,6925,28760,3731,San Diego International Airport,3777,Bellingham International Airport,51742,1625762
